## OpenLLMetryによるLLMアプリケーションの計装

必要なライブラリをダウンロードする。

In [ ]:
%pip install -r ../requirements.txt

Vector Storeの初期化を行う

In [ ]:
import os, logging
from dotenv import load_dotenv, find_dotenv

from langchain_openai import OpenAIEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import CSVLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

logger = logging.getLogger(__name__)

_ = load_dotenv(find_dotenv())
openai_api_key = os.getenv("OPENAI_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY")

model_provider_name = "cohere" # or openai

if model_provider_name == "openai":
    embeddings = OpenAIEmbeddings(api_key=openai_api_key)
elif model_provider_name == "cohere":
    embeddings = CohereEmbeddings(
        cohere_api_key=cohere_api_key, model="embed-multilingual-v3.0"
)
else:
    logger.error("Unsetted model name")

embeddings = OpenAIEmbeddings(api_key=openai_api_key)
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

documents = CSVLoader(file_path="../data/cndw2024_accepted_sessions.csv").load()

result = vector_store.add_documents(documents=documents)

計装対象のアプリケーションを実装する

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_cohere import ChatCohere
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from traceloop.sdk import Traceloop

# OTLP/GRPC
# os.environ["TRACELOOP_BASE_URL"] = "localhost:4317"
# OTLP/HTTP
os.environ["TRACELOOP_BASE_URL"] = "http://localhost:4318"

Traceloop.init(
    # デモ用なので、batch processorではなく即時でトレースデータを送る
    disable_batch=True,
    # アプリケーションの名前
    app_name="Ask the BigBaBy",
    # 独自属性の追加
    resource_attributes={
        "env": "demo",
        "version": "1.0.0"
    },
)
model_provider_name = "cohere" # or openai
temperature = 1.0
max_tokens = 2024

if model_provider_name == "openai":
    chat_model = ChatOpenAI(
        api_key=openai_api_key,
        model="gpt-4o-mini",
        temperature=temperature,
        max_tokens=max_tokens,
    )
elif model_provider_name == "cohere":
    chat_model = ChatCohere(
        cohere_api_key=cohere_api_key,
        model="command-r-plus",
        temperature=temperature,
        max_tokens=max_tokens,
    )
else:
    logger.error("Unsetted model name")

prompt = PromptTemplate.from_template(template="""
以下の質問をコンテキストに基づいて、答えてください。

## コンテキスト
{context}

## 質問
{question}
""")

chain = (
    {"question": RunnablePassthrough(), "context": vector_store.as_retriever()}
    | prompt
    | chat_model
    | StrOutputParser()
)

response = chain.invoke(input="Kubernetes関係でおすすめのセッションはありますか？")

print(response)